In [1]:
import pandas as pd
import sqlite3 

In [2]:
df = pd.read_csv('bom.movie_gross.csv.gz')

In [3]:
pd.options.display.float_format = '{:.2f}'.format

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [5]:
df.tail()

,title,studio,domestic_gross,foreign_gross,year
3382,The Quake,Magn.,6200.00,NaN,2018
3383,Edward II (2018 re-release),FM,4800.00,NaN,2018
3384,El Pacto,Sony,2500.00,NaN,2018
3385,The Swan,Synergetic,2400.00,NaN,2018
3386,An Actor Prepares,Grav.,1700.00,NaN,2018


In [6]:
df['foreign_gross'].fillna(0.0, inplace = True)

In [7]:
df['domestic_gross'].fillna(0.0, inplace = True)

In [8]:
df['foreign_gross'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 3387 entries, 0 to 3386
Series name: foreign_gross
Non-Null Count  Dtype 
--------------  ----- 
3387 non-null   object
dtypes: object(1)
memory usage: 26.6+ KB


In [9]:
df[df['foreign_gross'].str.contains(',') == True]

,title,studio,domestic_gross,foreign_gross,year
1872,Star Wars: The Force Awakens,BV,936700000.00,"1,131.6",2015
1873,Jurassic World,Uni.,652300000.00,"1,019.4",2015
1874,Furious 7,Uni.,353000000.00,"1,163.0",2015
2760,The Fate of the Furious,Uni.,226000000.00,"1,010.0",2017
3079,Avengers: Infinity War,BV,678800000.00,"1,369.5",2018


In [10]:
df['foreign_gross'] = df['foreign_gross'].str.replace(',','')

In [11]:
df['foreign_gross'] = df['foreign_gross'].astype(float)

In [12]:
df['total_gross'] = df['domestic_gross'] + df['foreign_gross']

In [13]:
df.head()

,title,studio,domestic_gross,foreign_gross,year,total_gross
0,Toy Story 3,BV,415000000.00,652000000.00,2010,1067000000.00
1,Alice in Wonderland (2010),BV,334200000.00,691300000.00,2010,1025500000.00
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.00,664300000.00,2010,960300000.00
3,Inception,WB,292600000.00,535700000.00,2010,828300000.00
4,Shrek Forever After,P/DW,238700000.00,513900000.00,2010,752600000.00


In [14]:
conn = sqlite3.connect('im.db.unzipped')

In [18]:
pd.read_sql('''
select *
from movie_basics
join movie_ratings
using (movie_id)
where averagerating >= 8;
''', conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt10356526,Laiye Je Yaarian,Laiye Je Yaarian,2019,117.00,Romance,8.30,31
1,tt10384606,Borderless,Borderless,2019,87.00,Documentary,8.90,559
2,tt1193623,Buried Prayers,Buried Prayers,2010,NaN,Documentary,8.00,5
3,tt1326743,California State of Mind: The Legacy of Pat Brown,California State of Mind: The Legacy of Pat Brown,2011,86.00,"Biography,Documentary",8.40,21
4,tt1403990,A Date with Fear,A Date with Fear,2011,90.00,"Comedy,Drama,Thriller",8.50,31
...,...,...,...,...,...,...,...,...
9443,tt9219848,Elon Musk: The Real Life Iron Man,Elon Musk: The Real Life Iron Man,2018,71.00,"Biography,Documentary",8.70,18
9444,tt9367004,Up to the Last Drop: The Secret Water War in E...,Mehri tin teleftaia stagona: O mystikos polemo...,2017,58.00,Documentary,8.20,5
9445,tt9590776,Delta Zoo,Delta Zoo,2019,NaN,Documentary,9.20,37
9446,tt9768966,Plugged in,Plugged in,2019,53.00,Documentary,8.60,27


In [57]:
pd.read_sql('''
select genres,count(genres), averagerating
from movie_basics
join movie_ratings
using (movie_id)
where averagerating >= 8
group by (genres)
''', conn)

,genres,count(genres),averagerating
0,None,0,8.50
1,Action,87,8.30
2,"Action,Adventure",4,8.50
3,"Action,Adventure,Animation",14,8.40
4,"Action,Adventure,Biography",5,8.00
...,...,...,...
455,Sport,25,8.30
456,Thriller,103,8.60
457,"Thriller,Western",1,8.10
458,War,8,9.20


In [78]:
pd.read_sql('''
select genres, count(genres), averagerating
from movie_basics
join movie_ratings
using (movie_id)
where genres = 'Action' and averagerating >= 8

''', conn)

,genres,count(genres),averagerating
0,Action,87,8.30
